## Importing the data

In [25]:
import pandas as pd
import numpy as np

In [26]:
# df = pd.read_json(r"../data/df_final_para.json") # paragraphs
df = pd.read_json(r"../data/df_final_document.json") # document
df = df[df['submit']==1] # only submitted

In [27]:
df.head()

,Reference,Feedback date,User type,Scope,Organisation name,Transparency register number,Organisation size,label_132,label_134,submit,file_name,language,text,text_clean
1,F550827,8.09.2020 23:54,NGO (Non-governmental organisation),None,Lie Detectors,094738529674-10,Small (< 50 employees),Yes,No,1,F550827-Lie_Detectors_-_Digital_Services_Act_-...,en,"\n \n \n8 Sept, 2020 \n \nDigital Services Ac...",sept digit servic act new competit tool lie de...
3,F550825,8.09.2020 23:45,Academic/Research Institution,None,Master student candidate Master in Competition...,None,Large (250 or more),Yes,No,1,F550825-PREPARATIVE_DOCUMENT_FOR_THE_EUROEPAN_...,en,Ref. Ares(2020)4723305 - 10/09/2020\n \n \n \n...,ref are 10/09/2020 the lego monopoli how platf...
7,F550814,8.09.2020 23:02,Company/Business organisation,None,SKY,62536168216-12,Large (250 or more),No,No,1,F550814-NCT_Response_-_Sky_September_2020.,en,Ref. Ares(2020)4723255 - 09/09/2020\n \n \n \n...,ref are 09/09/2020 the propos creat ‘ new comp...
9,F550811,8.09.2020 22:58,Consumer Organisation,None,Public Knowledge & The Consumer Federation of ...,None,Small (< 50 employees),Yes,No,1,F550811-CFA_PK_Additional_Background_.,en,Ref. Ares(2020)4723185 - 09/09/2020\n \n \n \n...,ref are 09/09/2020 onc recogn economi scale sc...
11,F550809,8.09.2020 22:39,Other,None,IMPALA - Independent Music Companies Association,12383069253-19,Micro (< 10 employees),Yes,No,1,F550809-IMPALA_Summary_-_consultation_on_a_new...,en,Ref. Ares(2020)4723184 - 09/09/2020\n \nCONFID...,ref are 09/09/2020 confidenti annex to impala ...


## Splitting the data

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label.132'] = le.fit_transform(df['label_132'])
df['label.134'] = le.fit_transform(df['label_134'])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
dfm = vectorizer.fit_transform(df['text_clean'])


In [56]:
from sklearn.model_selection import train_test_split

# split data into train, test, validation sets
train_text, test_text, train_labels, test_labels = train_test_split(dfm, df['label.132'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label.132'])

# val_text, test_text, val_labels, test_labels = train_test_split(temp_dfm, temp_labels, 
#                                                                 random_state=2018, 
#                                                                 test_size=0.5, 
#                                                                 stratify=temp_labels)

## XG-boost

In [59]:
# Xg_boost

from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=1000, max_depth=1000, learning_rate=0.1, objective='binary:logistic')

bst.fit(train_text, train_labels)

print(bst)

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support


y_pred = bst.predict(test_text)
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


# precision, recall, f_score, _ =  precision_recall_fscore_support(test_labels, y_pred, average= 'binary' )
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


precision = precision_score(test_labels, y_pred, average='binary')
recall = recall_score(test_labels, y_pred, average='binary')
f_score = 2 * (precision * recall) / (precision + recall)

print(f'\n Precision: {precision:.3f} \n Recall: {recall:.3f} \n  F1: {f_score:.3f}')

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=1000, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)
Accuracy: 54.55%

 Precision: 0.600 
 Recall: 0.500 
  F1: 0.545


## Logistic regression classifier

In [58]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(train_text, train_labels)

y_pred = clf.predict(test_text)

accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# precision, recall, f_score, _ =  precision_recall_fscore_support(test_labels, y_pred, average= 'binary' )
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


precision = precision_score(test_labels, y_pred, average='binary')
recall = recall_score(test_labels, y_pred, average='binary')
f_score = 2 * (precision * recall) / (precision + recall)


print(f'\n Precision: {precision:.3f} \n Recall: {recall:.3f} \n  F1: {f_score:.3f}')


Accuracy: 59.09%

 Precision: 0.636 
 Recall: 0.583 
  F1: 0.609
